# Télécom Paris - Kit Data Science - Session 4

1. **Parsing XML**
2. **Web scraping et parsing HTML**
3. **API**

### 1. Parsing XML (Extensible Markup Language)

Documentation : https://fr.wikipedia.org/wiki/Extensible_Markup_Language

Librairie lxml : https://lxml.de/tutorial.html

In [ ]:
# imports
import pandas as pd
from lxml import etree

# options d'affichage
pd.set_option("display.min_rows", 16)

**Exemple**

Les 150 propositions de la Convention Citoyenne pour le Climat

https://www.data.gouv.fr/fr/datasets/les-150-propositions-de-la-convention-citoyenne-pour-le-climat/

In [ ]:
# parsing XML
root = etree.parse('propositions.xml')

In [ ]:
# type
type(root)

In [ ]:
# trouver un élément
element = root.find('categorie')

In [ ]:
# type
type(element)

In [ ]:
# parent
element.getparent()

In [ ]:
# attribut
element.attrib['titre']

In [ ]:
# trouver un élément
element = root.find('categorie').find('sousCategorie').find('proposition')

In [ ]:
# text
element.text

In [ ]:
# nombre de catégories: getiterator
len([node for node in root.getiterator('categorie')])

In [ ]:
# nombre de sous-catégories: getiterator
len([node for node in root.getiterator('sousCategorie')])

In [ ]:
# nombre de propositions: getiterator
len([node for node in root.getiterator('proposition')])

In [ ]:
# print categorie / sousCategorie / proposition
for node_categorie in root.getiterator('categorie'):
    
    print(node_categorie.attrib['titre'])
    
    for node_sousCategorie in node_categorie.getiterator('sousCategorie'):
        
        print('> ', node_sousCategorie.attrib['titre'])
        
        for node_proposition in node_sousCategorie.getiterator('proposition'):
            
            print('>> ', node_proposition.text)

In [ ]:
# fabrication d'un DataFrame à partir d'une liste de dict
liste = [
        {'a': 1, 'b': 2},
        {'a': 3, 'b': 4},
        {'a': 5, 'b': 6, 'c': 7},
]

pd.DataFrame(liste)

In [ ]:
# fabrication d'un DataFrame à partir du XML
liste = [
    {'categorie': node_category.attrib['titre'],
    'sousCategorie': node_subcategory.attrib['titre'],
    'proposition': node_proposition.text}
        for node_category in root.getiterator('categorie')
            for node_subcategory in node_category.getiterator('sousCategorie')
                for node_proposition in node_subcategory.getiterator('proposition')]
liste

In [ ]:
# fabrication d'un DataFrame à partir du XML
liste = [
    {'categorie': node_category.attrib['titre'],
    'sousCategorie': node_subcategory.attrib['titre'],
    'proposition': node_proposition.text}
        for node_category in root.getiterator('categorie')
            for node_subcategory in node_category.getiterator('sousCategorie')
                for node_proposition in node_subcategory.getiterator('proposition')]

df = pd.DataFrame(liste)
df

In [ ]:
# catégories
df['categorie'].value_counts()

In [ ]:
# requêtes
df['proposition'].str.contains('biodiversité').sum()

In [ ]:
# requêtes
print(*df.loc[df['proposition'].str.contains('biodiversité'), 'proposition'], sep='\n')

In [ ]:
# requêtes
df['proposition'].str.contains('énergie').sum()

In [ ]:
# requêtes
print(*df.loc[df['proposition'].str.contains('énergie'), 'proposition'], sep='\n')

In [ ]:
# on stocke les propositions pour plus tard
propositions = df['proposition'].copy()

In [ ]:
# f(*args, **kwargs)

**Exercice 1**

Produire un DataFrame avec les colonnes : categorie, sousCategorie, oui (float).

**Exercice 2**

- Produire un DataFrame avec les colonnes : categorie, sousCategorie, oui (float), non (float), blancs (float).
- Calculer les sommes oui + non + blancs et oui + non &#9786;

**Exercice 3**

- Calculer un dictionnaire fréquentiel des mots  des propositions.
- Le mettre dans un objet de type Series trié par fréquences décroissantes.

In [ ]:
# re.findall()
import re
re.findall('[A-Za-zÀ-ÿ0-9]+', 'la fonction findall est très utile.')

In [ ]:
# Counter
from collections import Counter
c = Counter()
c.update(re.findall('[A-Za-zÀ-ÿ0-9]+', 'la fonction findall est très utile.'))
c.update(re.findall('[A-Za-zÀ-ÿ0-9]+', 'un objet Counter est aussi très utile.'))
c

In [ ]:
c = Counter()
var = propositions.apply(lambda x: re.findall('[A-Za-zÀ-ÿ0-9]+', x.lower()))
var.apply(c.update)
c

In [ ]:
from nltk.corpus import stopwords
stopwords_fr = stopwords.words('french')
stopwords_fr

In [ ]:
s = pd.Series(c)
s = s.drop(stopwords_fr, errors='ignore')
s = s.sort_values(ascending=False)
s

#### Validation XML / XSD

Il existe un langage de description de schéma XML appelé XSD (XML Schema Definition). Un fichier XML peut être écrit selon un schéma XSD particulier. Il existe une librairie Python qui peut vérifier qu'un fichier XML est valide selon un schéma XSD donné.

Librairie xmlschema :
https://pypi.org/project/xmlschema/

L'utilisation de la librairie est assez simple :

<pre>
>>> # vérification que le fichier "file.xml" est valide dans le schéma "schema.xsd"
>>> import xmlschema
>>> my_schema = xmlschema.XMLSchema('schema.xsd')
>>> #
>>> # retourne un booléen selon la validité
>>> my_schema.is_valid('file.xml')
</pre>

### 2. Web Scraping

- Extraction d'informations d'un site web.
- A utiliser en l'absence de données ouvertes ou d'API.
- Technique fragile car le site web peut changer du jour au lendemain.
- Problématique juridique...

**Avec requests**

Doc :
- requests : https://requests.readthedocs.io/en/master/

Installation :
- *pip install requests* ou *conda install -c anaconda requests*

Exemple de site : https://www.beerwulf.com/fr-fr

In [ ]:
import requests

r = requests.get('https://www.beerwulf.com/fr-fr')
r.status_code

In [ ]:
# content
r.content

In [ ]:
# type
type(r.content)

In [ ]:
# str en précisant un encodage
content = r.content.decode('utf-8')
content

In [ ]:
type(content)

#### Essai avec des regex

On cherche: `<span class="price">...</span>`

In [ ]:
# récupération mannuelle d'un prix avec une regex
# extraction de tous les caractères différents de <
# compris entre <span class="price"> et </span>
rx = re.compile('<span class="price">([^<]+)</span>')
match = rx.search(content)  # équivalent à match = re.search('<span class="price">([^<]+)</span>', content)
type(match)

In [ ]:
# extraction de niveau 0
match.group(0)

In [ ]:
# extraction de niveau 1
match.group(1)

In [ ]:
# récupération mannuelle de tous les prix avec une regex
for match in rx.finditer(content):
    print(match.group(1))

La technique est très fragile car elle s'appuie sur la syntaxe HTML exacte et non sur la sémantique...

In [ ]:
# récupération mannuelle de tous les prix avec une regex
rx = re.compile('<span class="price from-price strike-through">([^<]+)</span>')
for match in rx.finditer(content):
    print(match.group(1))

In [ ]:
# récupération mannuelle de tous les prix avec une regex
# ( from-price)? est une expression de capture
# possibilité d'utiliser (?: from-price)? qui n'est pas une expression de capture
rx = re.compile('<span class="price( from-price)?( strike-through)?">([^<]+)</span>')
for match in rx.finditer(content):
    print(match.group(1), match.group(2), match.group(3))

**Avec pandas.read_html()** recherche des tableaux dans les pages HTML

**Exemple**

Tableau page wikipédia: https://fr.wikipedia.org/wiki/Liste_des_pays_par_PIB_nominal

In [ ]:
# scraping d'une page HTML
var = pd.read_html("https://fr.wikipedia.org/wiki/Liste_des_pays_par_PIB_nominal")
[df.shape for df in var]

In [ ]:
# accès au n° 2
df = var[1]
df

In [ ]:
# accès à des valeurs
df.iloc[0]

In [ ]:
# index = columns du DataFrame
df.iloc[0].index

In [ ]:
# accès à des valeurs
df.iloc[[0, 1, 2, 76, -1]]

In [ ]:
# accès à une valeur
df.iloc[1, 2]

In [ ]:
df.iloc[-1, -1]

Chercher le code hexa \xa0 : https://www.codetable.net/hex/a0

In [ ]:
# aide sur read_html()
pd.read_html?

In [ ]:
# conversion automatique du séparateur des milliers
var = pd.read_html("https://fr.wikipedia.org/wiki/Liste_des_pays_par_PIB_nominal",
                    thousands='\xa0',
                    decimal=',')
df = var[1]
df.iloc[[0, 1, 2, 76, -1]]

In [ ]:
# reste à faire
df.loc[df['Pays ou territoire'].str.contains("[^A-Za-zÀ-ÿ0-9 \-']")]

Reste à faire :

Extraire les noms des pays sans les annotations.

**Avec beautifulsoup** parsing HTML

Doc :
- beautifulsoup : https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Installation :
- *pip install beautifulsoup4* ou *conda install -c anaconda beautifulsoup4*

In [ ]:
# imports
from bs4 import BeautifulSoup

**Exemple basique**

In [ ]:
html = """
<html>
    <head>
        <style>
        h1 { font-size: 50px; }
        body { font-family: Verdana; }
        li { color: red; }
        ul ul li { color: green; }
        .highlighted { font-weight: bold; }
        .italic { font-style: italic; }
        .highlighted.italic { }
        </style>
    </head>
    <body>
        <h1>Mon titre</h1>
        <p class="highlighted">
            Some text with a<br>
            <a href="https://google.com">link to google</a>
            <img src="https://picsum.photos/200/300">
        </p>
        <p>Some list:</p>
        <ul>
            <li>some item</li>
            <li class="highlighted italic">some item</li>
            <li class="italic">some item</li>
            <ul>
                <li>some other item 1</li>
                <li>some other item 2</li>
            </ul>
            <li>some item</li>
        </ul>
    </body>
</html>
"""

A tester sur : https://html.house

In [ ]:
# bs4
soup = BeautifulSoup(html)
soup

In [ ]:
# type
type(soup)

In [ ]:
# find h1
titre = soup.find('h1')
titre

In [ ]:
# type
type(titre)

In [ ]:
# name
titre.name

In [ ]:
# text
titre.text

In [ ]:
# find a
link = soup.find('a')
link

In [ ]:
# prochain tag
link.find_next()

In [ ]:
link.find_next().find_next()

In [ ]:
# attrs
link.attrs

In [ ]:
# text
link.text

In [ ]:
# find p
paragraph = soup.find('p')
paragraph

In [ ]:
# find img in paragraph
paragraph.find('img')

In [ ]:
# find_all
soup.find_all('li')

In [ ]:
# find_all
soup.find_all('li', {'class': "italic"})

In [ ]:
# idem avec un sélecteur css:
soup.select('li.italic')

In [ ]:
# Récupérer les li de 2e niveau qui sont dans un ul lui-même dans un ul
soup.find('ul').find('ul').find_all('li')

In [ ]:
# idem avec un sélecteur css:
soup.select('ul ul li')

In [ ]:
# accès au premier li
li = soup.select('ul ul li')[0]
li

In [ ]:
# prochain tag identique
li.find_next_sibling()

In [ ]:
# parent
li.parent

In [ ]:
# contents
li.parent.contents

In [ ]:
# que les tags
li.parent.find_all()

**Exemple 1**

Le Bon Coin

In [ ]:
# premier essai avec leboncoin

r = requests.get('https://www.leboncoin.fr/annonces/offres/ile_de_france/')
r

Codes erreurs du protocole HTTP : https://developer.mozilla.org/fr/docs/Web/HTTP/Status

In [ ]:
# contenu
r.content

In [ ]:
# en str
print(r.content.decode('utf-8'))

In [ ]:
# headers
headers = requests.utils.default_headers()
headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',
               'Accept-Language': 'fr,fr-FR;',})
headers

In [ ]:
# 2nd essai avec leboncoin
r = requests.get('https://www.leboncoin.fr/annonces/offres/ile_de_france/',
                 headers=headers)
r

**Exemple 2**

Craig List

In [ ]:
# essai avec craigslist
r = requests.get('https://paris.craigslist.org/d/locations-de-vacances/search/vac',
                 headers=headers)
r

In [ ]:
with open('Paris Locations de vacances - craigslist.htm', 'rb') as f:
    content = f.read()
content

In [ ]:
# BeautifulSoup
soup = BeautifulSoup(content)
soup

On utilise 2 méthodes :
    
- `find(tag, attrs)` : trouve le premier tag avec les attributs spécifiés
- `findAll(tag, attrs)` : trouve tous les tags avec les attributs spécifiés

In [ ]:
# exploration du HTML
# tag li avec class="result-row"

li_tag = soup.find('li', attrs={'class': 'result-row'})
print(li_tag)

In [ ]:
# type
type(li_tag)

La technique consiste par exemple à alimenter une liste de dictionnaires avec les valeurs trouvées pour chaque item et ensuite à le transformer en DataFrame :
- soit en utilisant `tag.attrs['attr']` pour collecter la valeur attr du tag <tag attr=value>
- soit en utilisant `tag.text` pour collecter la valeur <tag>text</tag>
- éventuellement en recherchant dans un nouveau tag à l'intérieur d'un tag donné

In [ ]:
# collecte des informations
# "data-pid"
# "time"
# "title"
# "price"
# "housing"
# "hood"
# "data-ids" (images)

rows = []

for li_tag in soup.findAll('li', attrs={'class': 'result-row'}):
    row = {}
    row['data-pid'] = li_tag.attrs['data-pid']
    t = li_tag.find('time')
    row['datetime'] = t.attrs['datetime']
    # à compléter
    rows.append(row)
    
rows

In [ ]:
# en DataFrame
df = pd.DataFrame(rows)
df

**Exercice 4**

Compléter le DataFrame (sauf images)

#### Inconvénients du web scraping:
- plutôt lent (car on parse potentiellement beaucoup de HTML inutile)
- ne donne pas les résultats attendus si une partie du contenu est intégré dynamiquement à la page via javascript
- un changement dans l'architecture du html ou du css (e.g: refonte du design du site) oblige à réécrire entièrement le programme

### API

Exemple: Deezer

Artiste : https://www.deezer.com/fr/artist/3037

Récupérer le nombre de fans d'un artiste avec requests :

In [ ]:
import requests
from bs4 import BeautifulSoup
# request
artist = 3037
response = requests.get(f'https://www.deezer.com/fr/artist/{artist}')
soup = BeautifulSoup(response.content)
nb_fans = int(soup.find('div', id='naboo_artist_social_small').span.text)
nb_fans

Récupérer le nombre de fans d'un artiste avec l'API :

Doc:
- https://pypi.org/project/deezer-python/

Installation :
- *pip install deezer-python*

Le terme "API" est très générique et peut désigner bien des choses, mais dans le jargon on l'utilise souvent pour désigner un service web qui renvoie non pas:
> des pages web au format HTML (destinées à être lues par un humain dans son navigateur)

mais:
> des données au format JSON (destinées à être traitées par un programme)

![img](https://miro.medium.com/max/4238/1*OcmVkcsM5BWRHrg8GC17iw.png)

Puisque les API sont dédiées à l'usage via des programmes, elles disposent en général d'une bonne documentation, et sont fiables et stables dans le temps. Tandis que sur des pages web HTML classiques, le design peut par exemple changer du jour au lendemain et rendre votre programme BeautifulSoup obsolète.

In [ ]:
# API JSON
response = requests.get(f'https://api.deezer.com/artist/{artist}')
data = response.json()
data

In [ ]:
# nb_fan
data['nb_fan']

In [ ]:
# picture
from IPython.display import Image
url = data['picture']
r = requests.get(url)
Image(data=r.content)

#### Avantages d'une API
- renvoie du format JSON, facile et rapide à traiter
- renvoie un format stable et documenté (voire versionné)
- exemple : https://developer.twitter.com/en/docs/twitter-api/api-reference-index
- la documentation indique comment interagir avec l'API:
    - quelle url
    - quelle méthode http (GET, POST, ...)
    - quels paramètres
    - ...
→ idéal pour les développeurs

### Quel intérêt pour le fournisseur d'API ?

En général il met en place des quotas de requêtes ou d'autres limitations afin de proposer un service payant qui dispose de possibilités avancées / d'un meilleur support / etc.

C'est pourquoi de nombreux services nécessitent de se connecter avec son compte client pour utiliser une API (e.g. https://openweathermap.org/api)

#### **Basic Auth**

Exemple: accéder à https://kim.fspot.org/private/

Pour y accéder il est nécessaire d'utiliser les credentials suivant:
- login: admin
- password: secret

Si on ne les passe pas (ou si on ne passe pas les bons), on a une erreur 401 (= unauthorized).

In [ ]:
# sans login/password
res = requests.get('https://kim.fspot.org/private')
res

In [ ]:
# avec login password
res = requests.get('https://kim.fspot.org/private', auth=('admin', 'secret'))
res

In [ ]:
# contenu
res.content.decode('utf-8')

In [ ]:
# YAML pour masquer auth crendentials

#### Auth par token

Exemple sur openweathermap :
- documentation: https://openweathermap.org/appid
- mes tokens: https://home.openweathermap.org/api_keys

In [ ]:
# requête avec un token
token = ''

Avantage des tokens:
- évite que les requêtes HTTP contiennent le mot de passe - à la place elles contiennent un token
- si je me fais "voler" un token, je peux le supprimer de mon compte
- certains services fournissent des token plus ou moins limités : ainsi je peux accepter de prêter un token à quelqu'un d'autre si je sais qu'il ne pourra en faire qu'un usage restreint (e.g app facebook: voir mes infos de profil, pas publier des posts à ma place)

In [ ]:
# requête avec un token
url = f'http://api.openweathermap.org/data/2.5/weather?APPID={token}&q=Paris'
res = requests.get(url)

meteo = res.json()
meteo

In [ ]:
# type
type(meteo)

In [ ]:
# extractions
{'city': meteo['name'],
'country': meteo['sys']['country'],
'date': meteo['dt'],
'temp': meteo['main']['temp'] - 273.15,}

In [ ]:
# extractions
import time

{'city': meteo['name'],
'country': meteo['sys']['country'],
'date': time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(meteo['dt'])),
'temp': meteo['main']['temp'] - 273.15,}